In [17]:
import pandas as pd
import numpy as np
import datetime as dt

In [18]:
df_ride = pd.read_csv("C:\\Maestria Gaby\\Materia1\\LimpiezaDatosPython\\ride_sharing_new.csv")
#veamos que tiene el dataset
df_ride.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ride_id          76 non-null     int64 
 1   duration         76 non-null     object
 2   station_A_id     76 non-null     int64 
 3   station_A_name   76 non-null     object
 4   station_B_id     76 non-null     int64 
 5   station_B_name   76 non-null     object
 6   bike_id          76 non-null     int64 
 7   user_type        76 non-null     int64 
 8   user_birth_year  76 non-null     int64 
 9   user_gender      76 non-null     object
 10  tire_sizes       76 non-null     int64 
 11  ride_date        76 non-null     object
dtypes: int64(7), object(5)
memory usage: 7.2+ KB


In [19]:
#utilizamos describe para ver el resumen estadistico de los datos de la columna user_type
df_ride['user_type'].describe()

count    76.000000
mean      1.881579
std       0.692187
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       3.000000
Name: user_type, dtype: float64

In [11]:
#podemos a cambiar el tipo de dato con astype, y se lo asignamos a la nueva columna
df_ride['user_type'] = df_ride['user_type'].astype('category')

#Para asegurarnos que la conversion fue correcta utilizamos la declaracion assert
assert df_ride['user_type'].dtype == 'category'

#y nuevamente observamos las estadisticas resumidas de la nueva columna, y podemos ver que
#el resultado cambia al tratarse de una variable categorica
print(df_ride['user_type'].describe())

count     76
unique     3
top        2
freq      39
Name: user_type, dtype: int64


In [12]:
#eliminamos la cadena minutes con el metodo strip 
df_ride['duration'] = df_ride['duration'].str.strip('minutes')

#convertirmos duration a tipo int 
df_ride['duration'] = df_ride['duration'].astype('int')

#Para asegurarnos que la conversion fue correcta utilizamos la declaracion assert
assert df_ride['duration'].dtype == 'int'

#Finalmente conocemos cual es la media de la columna duration_time
print(df_ride['duration'].mean())

12.776315789473685


In [13]:
#usamos loc para seleccionar todo esos registros mayores a 27 establecerle el limite superior que es 27
df_ride.loc[df_ride['tire_sizes'] > 27, 'tire_sizes'] = 27
df_ride.head(10)

#convetirmos a tipo categorico
df_ride['tire_sizes'] = df_ride['tire_sizes'].astype('category')

df_ride['tire_sizes'].describe()

count     76
unique     2
top       27
freq      73
Name: tire_sizes, dtype: int64

In [14]:
#convertimos a ride_date en tipo de dato datetime
infer_datetime_format=True
df_ride['ride_date'] = pd.to_datetime(df_ride['ride_date']).dt.date

#almacenamos la fecha de hoy
today = dt.date.today()

# y por ultimo le aplicamos la fecha de hoy almacenada en today a aquellos registros con
#fechas futuras, es decir con fechas superiores a hoy
df_ride.loc[df_ride['ride_date'] > today, 'ride_date'] = today

#mostramos la fecha maxima
print(df_ride['ride_date'].max())

2022-04-05


C:\Users\Javier\AppData\Local\Temp\ipykernel_10924\3937866506.py:3: UserWarning: Parsing '19/01/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_ride['ride_date'] = pd.to_datetime(df_ride['ride_date']).dt.date
C:\Users\Javier\AppData\Local\Temp\ipykernel_10924\3937866506.py:3: UserWarning: Parsing '24/10/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_ride['ride_date'] = pd.to_datetime(df_ride['ride_date']).dt.date
C:\Users\Javier\AppData\Local\Temp\ipykernel_10924\3937866506.py:3: UserWarning: Parsing '21/01/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_ride['ride_date'] = pd.to_datetime(df_ride['ride_date']).dt.date
C:\Users\Javier\AppData\Local\Temp\ipykernel_10924\3937866506.py:3: UserWarning: Parsing '22/01/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consis

In [15]:
# Aqui buscamos duplicados 
duplicates = df_ride.duplicated('ride_id', keep = False)

# ordenamos por la columna ride_id los registros duplicados obtenidos
df_ride = df_ride[duplicates].sort_values( by = 'ride_id')

# y por ultimo imprimimos los datos duplicados, mostramos 
#en cuales columnas estaban estos registros
print(df_ride[['ride_id','duration','user_birth_year']])

    ride_id  duration  user_birth_year
21       33        11             1986
38       33         6             1966
52       55        13             1999
64       55        10             1999
72       71        11             1987
73       71        11             1987
74       89         9             1990
75       89         9             1990


In [16]:
# Eliminamos los duplicados completos en ride_sharing
ride_dup = df_ride.drop_duplicates()

# Creamos un diccionario de estadisticas para funciones de agregacion
statistics = {'user_birth_year': 'min', 'duration': 'mean'}

# Agrupamos por ride_id y calculamos las nuevas estadisticas
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# nuevamente buscamos valores duplicados
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# y por ultimo nos aseguramos que se haya aplicado todo correctamente
assert duplicated_rides.shape[0] == 0